# BEST Processor Retry Data Processing

In [1]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.243:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 40g
--executor-memory 29g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=420000
--conf spark.shuffle.registration.timeout=42000
--conf spark.driver.cores=4
--conf spark.executor.cores=8
--conf spark.driver.maxResultSize=38
pyspark-shell
"""

# --driver-memory 70g
# --executor-memory 25g
# --conf spark.executor.heartbeatInterval=42000
# --conf num-executors=4
# --conf spark.shuffle.service.enabled=true
# --conf spark.dynamicAllocation.executorIdleTimeout=42000
# --conf spark.dynamicAllocation.enabled=true
# --conf spark.dynamicAllocation.minExecutors=4

import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="BestProcessorRetry")

env: SPARK_HOME=/opt/spark


In [2]:
sc._conf.set("spark.sql.execution.arrow.enabled", "true")
# sc._conf.set('spark.driver.maxResultSize', '32')
# sc._conf.set('spark.driver.cores', '4')
# sc._conf.set('spark.executor.cores', '7')
# # sc._conf.set('spark.driver.memory', '24g')
# # sc._conf.set('spark.executor.memory', '24g')
# sc._conf.set("spark.cores.max", 6)
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.network.timeout", "1000000s")
# sc._conf.set("spark.executor.heartbeatInterval", "1000000s")

# sc._conf.set("spark.cassandra.input.split.size_in_mb", 32)
# sc._conf.set("spark.cassandra.splitCount", 16)
# sc._conf.set("spark.cassandra.connection.host", "10.224.12.32")

# sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
# sc._conf.set("spark.cassandra.auth.username", "mlprw")
# sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.set("spark.cassandra.connection.host", "10.47.65.95")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "Df^9c2%znt7S")

sc._conf.getAll()

[('spark.driver.cores', '4'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.cassandra.auth.password', 'Df^9c2%znt7S'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.maxResultSize', '38'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:///root/.ivy2/jars/net.razor

In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.243:7077").\
    appName("BestProcessorRetry").\
    config(conf=sc._conf).\
    getOrCreate()
# spark

sqlContext = SQLContext(sc)
sqlContext

In [13]:
import itertools

# Test#1: Merge failed processor attempt with the next processor attempt
def merge_by_processor_attempt_1(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
    
    
        
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    print(initial_att_num)
    print(last_att_num)
    print('-----------')
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num) +1):

        indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
        
        print(indexes)
        if len(indexes) > 1:
        
            for i in range(0, len(indexes)-1):
                merged = sorted_group.loc[indexes[i+1]].copy(deep=True)
                initial_row = sorted_group.loc[indexes[i]]

                failed_attempt_date = initial_row['transaction_date_in_string']                   
                merged['failed_attempt_date'] = failed_attempt_date

                merged['failed_day_of_month'] = initial_row['day_of_month']
                merged['day_of_month'] = int(merged['day_of_month'])
                merged['failed_response_code'] = initial_row['response_code']

                failed_response_msg = initial_row['response_message']
                merged['failed_response_message'] = failed_response_msg  

                merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
                merged['days_between'] = days_between_ds(merged)
                merged['failed_event_uuid'] =  initial_row['event_uuid']   
                new_group = new_group.append(merged)
        
           
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [7]:
import itertools
import time
import calendar, datetime
# from datetime import timedelta
# Test#2: Merge failed processor attempt with the last processor attempt on the same day

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

def to_day(datestr):
    """Converts date string to day, e.g: '2018-05-02 00:00:00' to be '2' """
    return to_date(datestr).day

def merge_by_processor_attempt_2(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    sorted_group = group.sort_values(by=['transaction_date', 'cid'], na_position='first')
    
    
        
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    print(initial_att_num)
    print(last_att_num)
    print('---Test #2 -----')
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num) +1):

        indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
#         indexes = sorted_group[sorted_group['renew_att_num'] == (att_num)].index.tolist()
        
        print(indexes)
        if len(indexes) > 1:
            last_attempt = sorted_group.loc[indexes[-1]]
            for i in range(0, len(indexes)-1):
                merged = last_attempt.copy(deep=True)
                initial_row = sorted_group.loc[indexes[i]]

                failed_attempt_date = initial_row['transaction_date_in_string']                   
                merged['failed_attempt_date'] = failed_attempt_date

#                 merged['failed_day_of_month'] = initial_row['day_of_month']
                merged['failed_day_of_month'] = int(to_day(initial_row['transaction_date_in_string']))
                merged['day_of_month'] = int(to_day(merged['transaction_date_in_string']))
#                 merged['day_of_month'] = int(merged['day_of_month'])
                merged['failed_response_code'] = initial_row['response_code']

                failed_response_msg = initial_row['response_message']
                merged['failed_response_message'] = failed_response_msg  

                merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
                merged['days_between'] = days_between_ds(merged)
                merged['failed_event_uuid'] =  initial_row['event_uuid']   
                new_group = new_group.append(merged)
        
           
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [4]:
import itertools
import time
import calendar, datetime
# from datetime import timedelta
# Test#3: Merge failed processor attempt with the success attempt if any else merge with last processor attempt on the same day

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

def to_day(datestr):
    """Converts date string to day, e.g: '2018-05-02 00:00:00' to be '2' """
    return to_date(datestr).day


def processor_mid_changed(initial_row, merged_row):
    if initial_row['payment_service_id'] == merged_row['payment_service_id']:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_and_mid_unchanged"
        else:
            return "processor_unchanged_mid_changed"
    else:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_changed_mid_unchanged"
        else:
            return "processor_changed_mid_changed"
        
    

def merge_by_processor_attempt(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''
    
    group['failed_cc_expiration_date'] = ''
    group['expiration_date_changed'] = False
    group['processor_mid_changed'] = '' 
    group['failed_cid'] = '' 
    
    new_group = pd.DataFrame(columns = group.columns)
    sorted_group = group.sort_values(by=['transaction_date', 'cid'], na_position='first')
    
    sorted_index = sorted_group.index.tolist()
    successful_df = sorted_group[sorted_group['success'] == 1]
        
    if not successful_df.empty:
        first_succes_index = successful_df.index[0]
        sorted_group_index = sorted_index[:sorted_index.index(first_succes_index) +1]
        sorted_group = sorted_group[sorted_group.index.isin(sorted_group_index)]    
        
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]

    
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num) +1):

        indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
#         indexes = sorted_group[sorted_group['renew_att_num'] == (att_num)].index.tolist()
        
        if len(indexes) > 1:
            last_attempt = sorted_group.loc[indexes[-1]]
            for i in range(0, len(indexes)-1):
                merged = last_attempt.copy(deep=True)
                initial_row = sorted_group.loc[indexes[i]]

                failed_attempt_date = initial_row['transaction_date_in_string']                   
                merged['failed_attempt_date'] = failed_attempt_date

                merged['failed_day_of_month'] = int(to_day(initial_row['transaction_date_in_string']))
                merged['day_of_month'] = int(to_day(merged['transaction_date_in_string']))
                merged['failed_response_code'] = initial_row['response_code']

                failed_response_msg = initial_row['response_message']
                merged['failed_response_message'] = failed_response_msg  

                merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
                merged['days_between'] = days_between_ds(merged)
                merged['failed_event_uuid'] =  initial_row['event_uuid']
                
                
                ###### Below are additional fields for processor retry ########
                merged['failed_cc_expiration_date'] = initial_row['cc_expiration_date']
#                 merged['expiration_date_changed'] = str(initial_row['cc_expiration_date'] is not merged['cc_expiration_date'])

                initial_cc = initial_row['cc_expiration_date'].replace('/', '')
                merged_cc = merged['cc_expiration_date'].replace('/', '')
                merged['expiration_date_changed'] = (initial_cc != merged_cc)

                merged['processor_mid_changed'] = processor_mid_changed(initial_row, merged)
                merged['failed_cid'] = initial_row['cid']
                ##############################
                new_group = new_group.append(merged)
        
           
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [26]:
import itertools
import time
import calendar, datetime
# from datetime import timedelta
# Test#3: Merge failed processor attempt with the success attempt if any else merge with last processor attempt on the same day

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

def to_day(datestr):
    """Converts date string to day, e.g: '2018-05-02 00:00:00' to be '2' """
    return to_date(datestr).day


def processor_mid_changed(initial_row, merged_row):
    if initial_row['payment_service_id'] == merged_row['payment_service_id']:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_and_mid_unchanged"
        else:
            return "processor_unchanged_mid_changed"
    else:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_changed_mid_unchanged"
        else:
            return "processor_changed_mid_changed"

        
def processor_att_num(cid: str):
    if not cid or cid == 'FIRST_ATTEMPT':
        return 1
    elif cid == 'RETRY_DECLINED.2' or cid == 'RETRY_FAILED.2':
        return 2
    elif cid == 'RETRY_DECLINED.3' or cid == 'RETRY_FAILED.3':
        return 3
    elif cid == 'RETRY_DECLINED.4' or cid == 'RETRY_FAILED.4':
        return 4
    elif cid == 'RETRY_DECLINED.5' or cid == 'RETRY_FAILED.5':
        return 5
    
    return -1  
    

def merge_by_processor_attempt_new(group):

    
    #Assign default values to additonal columns
#     group['failed_attempt_date'] = ''
#     group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
#     group['days_between'] = 0
    group['failed_event_uuid'] = ''
    
    group['failed_cc_expiration_date'] = ''
    group['expiration_date_changed'] = False
    group['processor_mid_changed'] = '' 
    group['failed_cid'] = ''
    group['failed_payment_service_id'] = ''
    group['failed_merchant_number'] = ''
    group['processor_att_num'] = 0 
    
    new_group = pd.DataFrame(columns = group.columns)
    sorted_group = group.sort_values(by=['transaction_date', 'cid'], na_position='first')
    
    sorted_index = sorted_group.index.tolist()
    successful_df = sorted_group[sorted_group['success'] == 1]
        
    if not successful_df.empty:
        first_succes_index = successful_df.index[0]
        sorted_group_index = sorted_index[:sorted_index.index(first_succes_index) +1]
        sorted_group = sorted_group[sorted_group.index.isin(sorted_group_index)]    
        
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]

    
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num) +1):

        indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
#         indexes = sorted_group[sorted_group['renew_att_num'] == (att_num)].index.tolist()
        
        if len(indexes) > 1:
            last_attempt = sorted_group.loc[indexes[-1]]
            for i in range(0, len(indexes)-1):
                merged = last_attempt.copy(deep=True)
                initial_row = sorted_group.loc[indexes[i]]

#                 failed_attempt_date = initial_row['transaction_date_in_string']                   
#                 merged['failed_attempt_date'] = failed_attempt_date

#                 merged['failed_day_of_month'] = int(to_day(initial_row['transaction_date_in_string']))
                merged['day_of_month'] = int(to_day(merged['transaction_date_in_string']))
                merged['failed_response_code'] = initial_row['response_code']

                failed_response_msg = initial_row['response_message']
                merged['failed_response_message'] = failed_response_msg  

                merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
#                 merged['days_between'] = days_between_ds(merged)
                merged['failed_event_uuid'] =  initial_row['event_uuid']
                
                
                ###### Below are additional fields for processor retry ########
                merged['failed_cc_expiration_date'] = initial_row['cc_expiration_date']
#                 merged['expiration_date_changed'] = str(initial_row['cc_expiration_date'] is not merged['cc_expiration_date'])

                initial_cc = initial_row['cc_expiration_date'].replace('/', '')
                merged_cc = merged['cc_expiration_date'].replace('/', '')
                merged['expiration_date_changed'] = (initial_cc != merged_cc)

                merged['processor_mid_changed'] = processor_mid_changed(initial_row, merged)
                merged['failed_cid'] = initial_row['cid']
                
                merged['failed_payment_service_id'] = initial_row['payment_service_id']
                merged['failed_merchant_number'] = initial_row['merchant_number']
                
                merged['processor_att_num'] = processor_att_num(merged['cid'])
                ##############################
                new_group = new_group.append(merged)
        
           
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [22]:
import time

import pandas as pd
import numpy as np

from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = {  
             'do_not_honor' : 'do not honor', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try_merchant_review' : 'do not try again/merchant review',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'system error',
            'no reply' : 'no reply',
             'no charge model found' : 'no charge model found',
             'issuer unavailable' : 'issuer unavailable',
             'litle http response code' : 'litle http response code',
             'ioexception' : 'ioexception',
             'invalid merchant' : 'invalid merchant',
             'international filtering':'international filtering',
             'corrupt input data':'corrupt input data',
             'server error' : 'server error',
             'acquirer error' : 'acquirer error',
             'transaction refused[30]':'transaction refused[30]',
             'transaction refused[002]':'transaction refused[002]',
             'txn_refused' : 'refuse',
             'declined non generic': 'declined non generic',
             'declined' : 'decline', 
             'transaction not allowed at terminal': 'transaction not allowed at terminal',
             'error validating xml data': 'error validating xml data',
             'communication problems': 'communication problems',
             'new account info': 'new account info',
             'unable to connect to gateway': 'unable to connect to gateway'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-02' ")

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("failed_event_uuid", StringType())\
                            .add("failed_cc_expiration_date", StringType())\
                            .add("expiration_date_changed", BooleanType())\
                            .add("processor_mid_changed", StringType())\
                            .add("failed_cid", StringType())\
                            .add("failed_payment_service_id", StringType())\
                            .add("failed_merchant_number", StringType())\
                            .add("processor_att_num", IntegerType())
                            
                        
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_processor_attempt_new)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 64)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape  # (6733746, 53)

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(1930717, 57)

In [33]:
# pdf_all.to_csv(WORK_DIR + 'processor_retry_2019_01_to_03.csv')

In [32]:
pdf_all['received_date'].value_counts()

2019-01-02    26627
2019-01-25    25580
2019-01-10    25558
2019-01-12    25193
2019-03-21    25059
2019-02-01    24968
2019-01-17    24900
2019-01-18    24724
2019-02-04    24471
2019-01-28    24363
2019-01-03    24337
2019-01-05    24303
2019-01-04    24240
2019-01-22    24189
2019-03-10    24029
2019-02-09    23977
2019-02-08    23789
2019-01-24    23622
2019-02-27    23568
2019-02-07    23549
2019-01-20    23538
2019-02-06    23461
2019-03-14    23437
2019-01-16    23333
2019-02-05    23324
2019-02-02    23319
2019-01-27    23286
2019-03-28    23232
2019-01-26    23013
2019-03-07    22997
              ...  
2019-02-15    20009
2019-02-20    19888
2019-03-16    19816
2019-03-02    19737
2019-03-06    19703
2019-03-27    19653
2019-02-21    19616
2019-03-03    19602
2019-03-17    19588
2019-03-13    19534
2019-03-08    19432
2019-02-12    19340
2019-02-11    19293
2019-02-18    18933
2019-03-30    18818
2019-02-16    18585
2019-03-05    18529
2019-02-23    18512
2019-03-11    18469


In [28]:
pdf_all['processor_att_num'] = pdf_all[pdf_all['processor_att_num'] > -1]['processor_att_num'] + 1
pdf_all['processor_att_num'].value_counts()

2.0    1825150
3.0     105272
4.0        226
5.0         34
Name: processor_att_num, dtype: int64

In [31]:
# pdf_all['processor_att_num'] = pdf_all['processor_att_num'].fillna(-1)
# pdf_all['processor_att_num'].value_counts()
pdf_all['processor_att_num'] = pdf_all['processor_att_num'].astype(int)
pdf_all['processor_att_num'].value_counts()

 2    1825150
 3     105272
 4        226
-1         35
 5         34
Name: processor_att_num, dtype: int64

In [82]:
# pdf_all.to_csv(WORK_DIR + 'processor_retry_01_to_12.csv')
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
pdf_all = pd.read_csv(WORK_DIR + 'processor_retry_01_to_12.csv')
pdf_all.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(6733746, 57)

In [25]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_response_code,failed_response_message,failed_decline_type,failed_event_uuid,failed_cc_expiration_date,expiration_date_changed,processor_mid_changed,failed_cid,failed_payment_service_id,failed_merchant_number,processor_att_num
0,2019-02-03,avast,10007475401,8230718493,2019-02-03 14:38:20,2,NON3DS,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediat...,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0420,RETRY_DECLINED.2,2,3,sunday,af5fedf6-62f6-489c-8fea-4a72b5c2e97b,1872543,Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,114.70,EUR,MasterCard,netgiro-bms,1,99.99,114.70,EUR,57531456100,27050,Do not try again/use alternate payment card.[6...,New,13274386200,2019-02-03 08:38:20,08,Authorize,0,530,Refused: Refused,txn_refused,d9683bf6-9d95-416e-9e46-a02a7b237bb0,0420,False,processor_changed_mid_changed,,adyen,DR_IRELAND,1
1,2019-02-08,avast,10007475401,8258737853,2019-02-08 12:15:40,0,NON3DS,CREDIT AGRICOLE S.A.,FR,513770,MasterCard,MasterCard Standard (Unembossed) Card Immediat...,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0418,RETRY_DECLINED.2,0,8,friday,840199bd-00a6-426d-aa44-e4a155f48067,1872543,Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,99.99,113.44,EUR,MasterCard,netgiro-bms,2,99.99,113.44,EUR,57730537600,27050,Do not try again/use alternate payment card.[6...,New,13274386200,2019-02-08 06:15:40,06,Authorize,0,522,05: Do not honor,do_not_honor,efae77a9-9eba-47eb-8200-d3d235f2424a,0418,False,processor_changed_mid_changed,,adyen,DR_IRELAND,1
2,2019-03-07,adbehkr,10059085901,8421456683,2019-03-07 06:41:52,,NON3DS,"HYUNDAICARD CO., LTD.",KR,552290,MasterCard,World MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capabil...,0922,RETRY_DECLINED.2,,7,thursday,ade6ea11-11c1-4d9a-9e52-60bc4ed4c6fd,,Credit,KR,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_KOREA-ENTITY,11000.0,9.8,KRW,MasterCard,netgiro-bms,1,11000.0,9.8,KRW,58876377200,27050,Do not try again/use alternate payment card.[6...,New,18805274000,2019-03-07 00:41:52,00,Authorize,0,27001,Transaction refused[62] [000000] [] [],Hard Fraud,27c2e03c-15c5-4b43-a222-c40f65023c1f,0922,False,processor_changed_mid_changed,,netgiro-seb,1156962062,1
3,2019-01-11,adbehkr,10076134701,8095353453,2019-01-11 06:40:54,,NON3DS,"Shinhan Card Co., Ltd.",KR,451842,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0221,RETRY_DECLINED.2,,11,friday,ce3cb7bb-0bdc-4552-9485-fb2f978cd68b,,Credit,KR,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_KOREA-ENTITY,11000,9.843372,KRW,Visa,netgiro-bms,1,11000,9.843372,KRW,56551670600,27001,Transaction refused[05] [] [111800] [111800],New,18181256200,2019-01-11 00:40:54,00,Authorize,0,27001,Transaction refused[05] [000000] [] [],txn_refused,2faa2dd5-13e0-4f30-9daa-ae74bafb81f2,0221,False,processor_changed_mid_changed,,netgiro-seb,1156962062,1
4,2019-01-18,adbehkr,10076134701,8137731603,2019-01-18 06:52:14,,NON3DS,"Shinhan Card Co., Ltd.",KR,451842,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0221,RETRY_DECLINED.2,,18,friday,e7dcfd38-065f-4a5e-8e89-8a7db13a55c4,,Credit,KR,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_KOREA-ENTITY,11000,9.788237,KRW,Visa,netgiro-bms,2,1100

In [51]:
pdf_all_07[pdf_all_07['subsegment_id'] == 16589929800].sort_values(by=['transaction_date', 'cid'], na_position='first')

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
805061,27539,2018-07-18,adbehru,10690088901,7074772863,2018-07-18 06:51:34,NaN,NON3DS,QIWI Bank (JSC),RU,489049.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0819,NaN,NaN,18,Wednesday,4fdcec9c-c2f4-4eb5-b4f5-bdbe046ec712,NaN,Prepaid,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,759.92,12.156967,RUB,Visa,netgiro-seb,1.0,759.92,12.156967,RUB,50033222000,27001,Transaction refused[51] [000000] [] [],New,16589929800,2018-07-18 01:51:34,1,Authorize,0.0
493400,26855,2018-07-18,adbehru,10690088901,7074773153,2018-07-18 06:51:36,NaN,NON3DS,QIWI Bank (JSC),RU,489049.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0819,RETRY_DECLINED.2,NaN,18,Wednesday,9394e8f2-7113-4414-869e-e7024a97daa2,NaN,Prepaid,RU,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,759.92,12.156967,RUB,Visa,netgiro-bms,1.0,759.92,12.156967,RUB,50033222000,27083,Insufficient Funds[51] [] [111800] [111800],New,16589929800,2018-07-18 01:51:36,1,Authorize,0.0
325,325,2018-07-25,adbehru,10690088901,7109875733,2018-07-25 06:52:12,NaN,NON3DS,QIWI Bank (JSC),RU,489049.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0819,NaN,NaN,25,Wednesday,50e6a8ce-f4d6-490e-8f2a-7188c657675b,NaN,Prepaid,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,759.92,12.112051,RUB,Visa,netgiro-seb,2.0,759.92,12.112051,RUB,50254299000,0,SUCCESS[00] [61JA1N] [] [],New,16589929800,2018-07-25 01:52:12,1,Authorize,1.0


In [38]:
pdf_all_07[pdf_all_07['subsegment_id'] == 16704354500].sort_values(by=['transaction_date', 'cid'], na_position='first')[['renew_att_num', 'event_uuid', 'transaction_date','cid', 'success']]

# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701']

,renew_att_num,event_uuid,transaction_date,cid,success
2231126,1.0,3b7174fe-651c-42dd-a619-40ece4fe768a,2018-07-17 10:50:07,NaN,0.0
2455349,1.0,79993804-ffda-4906-bf84-2db73f5fd4fc,2018-07-17 10:50:08,RETRY_DECLINED.2,0.0
1507515,1.0,be1278a7-0650-472f-aa61-7e4b2172a9a7,2018-07-17 10:50:08,RETRY_DECLINED.3,0.0
2453004,2.0,1846ba4d-3b16-404b-a40b-55740885b8d8,2018-07-25 16:45:24,NaN,0.0
2371589,2.0,868da96d-9e09-4106-96ea-9a6215af488c,2018-07-25 16:45:25,RETRY_DECLINED.2,0.0
22,2.0,18523094-31e6-4f4c-af07-85555798bd5b,2018-07-25 16:45:26,RETRY_DECLINED.3,0.0


In [40]:
pdf_all_07[pdf_all_07['subsegment_id'].isin([16704354500])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_processor_attempt)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success','failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between', 'transaction_date', 'day_of_month']]

event_uuid  \
subscription_id subsegment_id                                                 
11950475601     16704354500   1507515  be1278a7-0650-472f-aa61-7e4b2172a9a7   
                              1507515  be1278a7-0650-472f-aa61-7e4b2172a9a7   
                              22       18523094-31e6-4f4c-af07-85555798bd5b   
                              22       18523094-31e6-4f4c-af07-85555798bd5b   

                                                          failed_event_uuid  \
subscription_id subsegment_id                                                 
11950475601     16704354500   1507515  3b7174fe-651c-42dd-a619-40ece4fe768a   
                              1507515  79993804-ffda-4906-bf84-2db73f5fd4fc   
                              22       1846ba4d-3b16-404b-a40b-55740885b8d8   
                              22       868da96d-9e09-4106-96ea-9a6215af488c   

                                       renew_att_num received_date  success  \
subscription_id subsegment_id                                                 
11950475601     16704354500   1507515            1.0    2018-07-17      0.0   
                              1507515            1.0    2018-07-17      0.0   
                              22                 2.0    2018-07-25      0.0   
                              22                 2.0    2018-07-25      0.0   

                                      failed_response_code  \
subscription_id subsegment_id                                
11950475601     16704354500   1507515                  530   
                              1507515                  530   
                              22                       530   
                              22                       530   

                                      failed_response_message  \
subscription_id subsegment_id                                   
11950475601     16704354500   1507515       005: Do not honor   
                              1507515       349: Do Not Honor   
                              22            005: Do not honor   
                              22            349: Do Not Honor   

                                      failed_decline_type days_between  \
subscription_id subsegment_id                                            
11950475601     16704354500   1507515                                0   
                              1507515                                0   
                              22                                     0   
                              22                                     0   

                                          transaction_date day_of_month  
subscription_id subsegment_id                                            
11950475601     16704354500   1507515  2018-07-17 10:50:08           17  
                              1507515  2018-07-17 10:50:08           17  
                              22       2018-07-25 16:45:26           25  
                              22       2018-07-25 16:45:26           25

In [6]:
# pdf_all_07[pdf_all_07['subsegment_id'].isin([13865645800, 16460118700, 11815740300])].groupby(["subscription_id", "subsegment_id", "renew_att_num"]).apply(merge_by_sub)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between']]
# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701'].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)
# pdf_all_07[pdf_all_07['subsegment_id'].isin([13714616000, 13865645800])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_processor_attempt)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success','transaction_date_in_string', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between', 'transaction_date', 'day_of_month']]
pdf_all_07[pdf_all_07['subsegment_id'].isin([16203155800])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_processor_attempt)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success','transaction_date_in_string', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between', 'transaction_date', 'day_of_month']]

4.0
4.0
---Test #3 -----
[849598]
4.0
4.0
---Test #3 -----
[849598]


,,,event_uuid,failed_event_uuid,renew_att_num,received_date,success,transaction_date_in_string,failed_attempt_date,failed_response_code,failed_response_message,failed_decline_type,days_between,transaction_date,day_of_month
subscription_id,subsegment_id,,,,,,,,,,,,,,
11078418901,16203155800,849598,,,4.0,2018-07-04,1.0,2018-07-04 02:18:31,,,,,0,2018-07-04 07:18:31,4


In [109]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-12-10,avast,10019477901,7898877723,2018-12-10 15:45:46,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,,,10,monday,0355f182-0fee-44f3-99f6-27dfdf961bea,,Deferred Debit,IT,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,netgiro-bms,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[05] [] [111800] [111800],New,13296142400,2018-12-10 09:45:46,09,Authorize,0,2018-12-05 07:45:47,5,27001,Transaction refused[null] [null] [null] [null],txn_refused,5,a439cb8c-c234-488b-8a4a-f96a65cc7d19
1,2018-12-10,avast,10019477901,7898878403,2018-12-10 15:45:47,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,RETRY_DECLINED.2,,10,monday,b17b54ca-6935-423a-9a66-b3c3221466fd,,Deferred Debit,IT,DRI*AVAST Software,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,drwp-fd,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[null] [null] [null] [null],New,13296142400,2018-12-10 09:45:47,09,Authorize,0,2018-12-05 07:45:47,5,27001,Transaction refused[null] [null] [null] [null],txn_refused,5,a439cb8c-c234-488b-8a4a-f96a65cc7d19
2,2018-12-10,avast,10019477901,7898877723,2018-12-10 15:45:46,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,,,10,monday,0355f182-0fee-44f3-99f6-27dfdf961bea,,Deferred Debit,IT,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,netgiro-bms,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[05] [] [111800] [111800],New,13296142400,2018-12-10 09:45:46,09,Authorize,0,2018-12-05 07:45:48,5,27001,Transaction refused[05] [] [111800] [111800],txn_refused,5,c98e3a1c-8cae-406a-8cb6-47d108babf94
3,2018-12-10,avast,10019477901,7898878403,2018-12-10 15:45:47,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,RETRY_DECLINED.2,,10,monday,b17b54ca-6935-423a-9a66-b3c3221466fd,,Deferred Debit,IT,DRI*AVAST Software,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,drwp-fd,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[null] [null] [null] [null],New,13296142400,2018-12-10 09:45:47,09,Authorize,0,2018-12-05 07:45:48,5,27001,Transaction refused[05] [] [111800] [111800],txn_refused,5,c98e3a1c-8cae-406a-8cb6-47d108babf94
4,2018-12-12,adbehap,10051777801,7911511643,2018-12-12 06:37:33,,NON3DS,BOC Credit Card (International) Limited,HK,434662,Visa,"No Data Found', value: 'N/A",Business,Credit - True credit (No PIN/Signature capabil...,0521,,,12,wednesday,29caa62e-2111-4922-bd29-7fda12090546,,Credit,HK,DRI*Adobe Sales,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,108.0,13.8,HKD,Visa,netgiro-bms,2,108.0,13.8,HKD,55346067900,0,SUCCESS[00] [096863] [111800] [111800],New,17803489900,2018-12-12 00:37:33,00,Authorize,1,2018-12-05 00:36:37,5,27001,Transaction refused[05] [] [

In [17]:
# Merge only the last two renew_att_num with the last single attempt
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



# Merge only the last two renew_att_num with the last single attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    if df_last_attempt['renew_att_num']:
        last_att_num = int(df_last_attempt['renew_att_num'])
        previous_att_num = last_att_num - 1
    else:
        group['event_uuid'] = '' 
        return group
   
    df_failed_txn = group[(group['renew_att_num'].isin([ str(previous_att_num),  df_last_attempt['renew_att_num'] ])) & (group['event_uuid'] != df_last_attempt['event_uuid']) ]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = 0 #last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

pdf_all = toPandas(sdf_filtered, 64)  
# pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(6905996, 53)

In [21]:
bin_profile = pd.read_csv(WORK_DIR + 'bin_profile.csv')
bin_profile.shape



(32272, 11)

In [22]:
bin_profile['bin'] = bin_profile['bin'].apply(str).str.replace('.0', '', regex=False)
bin_profile.head()

,Unnamed: 0,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,0,nan,18.043255,0.011639,410.000000,27.498705,79.410000,102.195200,4.590894,4.590894,3579
1,5,352818,19.118323,4.800000,51.479167,19.031746,45.503799,50.284093,4.800000,4.800000,7
2,7,352821,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,48.115695,5
3,8,352822,34.316981,18.714051,49.360468,10.081940,46.126243,48.713623,18.714051,18.714051,10
4,12,352831,39.732697,18.336803,68.206773,16.005322,62.103159,66.986050,18.336803,18.336803,9


In [ ]:
pdf_all = pd.merge(pdf_all, bin_profile[['bin','Mean','Min','Max','StdDev','Max_95','Max_99','Min_1', 'Min_10']], left_on='bin', right_on='bin', how='left')
pdf_all.shape

In [24]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_last_two_with_bin.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_last_two_with_bin_01_to_12.csv')
pdf_all.to_csv(WORK_DIR + 'retry_success_processor_attempt_with_bin_01_to_12.csv')

In [69]:
sc.stop()

In [84]:
gp = pdf_all.groupby(['failed_response_code'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 100]

success        success_rate
                         sum  count             
failed_response_code                            
150.0                    332    507    65.483235
27053.0                 2038   3605    56.532594
833.0                    510    918    55.555556
27034.0                  687   1323    51.927438
231.0                    149    292    51.027397
000                      231    457    50.547046
0.0                     6217  12427    50.028164
27053                     63    129    48.837209
27019                     54    126    42.857143
27002.0                  760   1774    42.841037
27019.0                 1256   3020    41.589404
225                       54    138    39.130435
233.0                    936   2458    38.079740
11002.0                 1558   4202    37.077582
233                       42    114    36.842105
204.0                   1155   3230    35.758514
203.0                    834   2346    35.549872
225.0                   1410   3993    35.311796
204                       45    128    35.156250
27020.0                  158    464    34.051724
238                       35    106    33.018868
241.0                    353   1079    32.715477
23002.0                   72    221    32.579186
11002                     61    193    31.606218
203                       40    130    30.769231
27033.0                  998   3323    30.033103
301.0                   1329   4664    28.494854
27026.0                  896   3434    26.092021
301                       35    191    18.324607
27032.0                    9    115     7.826087
...                      ...    ...          ...
27004.0                    1    584     0.171233
606                        9   5371     0.167567
253                        1    737     0.135685
27051.0                   87  82514     0.105437
402.0                      2   2552     0.078370
253.0                     14  18330     0.076378
521.0                     27  43263     0.062409
591.0                     17  32108     0.052946
27051                      1   3639     0.027480
501.0                      1   5754     0.017379
27054.0                    5  30856     0.016204
27083.0                    9  91873     0.009796
304.0                      1  17326     0.005772
502.0                      0   5623     0.000000
27083                      0   3767     0.000000
806.0                      0   2481     0.000000
603.0                      0   1808     0.000000
521                        0   1581     0.000000
591                        0   1094     0.000000
27054                      0    819     0.000000
304                        0    727     0.000000
27022.0                    0    495     0.000000
509.0                      0    413     0.000000
607.0                      0    357     0.000000
501                        0    273     0.000000
502                        0    214     0.000000
602.0                      0    200     0.000000
596.0                      0    182     0.000000
401.0                      0    135     0.000000
508.0                      0    128     0.000000

[82 rows x 3 columns]

In [96]:
pdf_all['failed_cid'].fillna('nan', inplace=True)

In [97]:
gp = pdf_all.groupby(['failed_cid'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 1]

success          success_rate
                     sum    count             
failed_cid                                    
payerID               38       76    50.000000
RETRY_FAILED.2      1619     7310    22.147743
RETRY_FAILED.3        31      146    21.232877
nan               116940  6288397     1.859615
RETRY_DECLINED.2    2154   437774     0.492035
RETRY_FAILED.4         0       28     0.000000
RETRY_DECLINED.3       0       15     0.000000

In [99]:
gp = pdf_all.groupby(['failed_response_code', 'failed_decline_type'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
result = df_gp[df_gp[('success', 'count')] > 1]
result.to_csv(WORK_DIR + 'processor_retry_2018_error_code_and_decline_type_success_rate.csv')
result

success        \
                                                           sum count   
failed_response_code      failed_decline_type                          
126                       unable to connect to gateway       4     5   
97                        Base                               3     4   
151                       invalid_cc                        54    79   
000                       no reply                          63    94   
530                       invalid merchant                   2     3   
150                       invalid_cc                       340   519   
0                         server error                    2171  3441   
301                       issuer unavailable              1255  1996   
0                         no reply                        1462  2335   
23006                     communication problems            33    53   
000                       server error                      84   136   
225                       unable to determine format       747  1283   
530                       invalid_cc                       102   178   
27053                     do_not_try_merchant_review      2101  3734   
833                       do_not_try_merchant_review       531   951   
131                       Base                               6    11   
27034                     system error                     717  1375   
231                       invalid merchant                 104   208   
000                       system error                      13    26   
225                       Base                               1     2   
204                       Base                             778  1561   
231                       Base                              48   102   
000                       Base                               4     9   
233                       invalid_cc                       977  2242   
0                         system error                     297   683   
27002                     server error                     794  1850   
522                       new account info                  56   131   
38001                     system error                       5    12   
27019                     invalid_cc                      1310  3146   
22054                     system error                       7    17   
...                                                        ...   ...   
606                       declined                           0   130   
508                       Base                               0   128   
27050                     Base                               0    90   
591                       Base                               0    69   
27048                     Hard Fraud                         0    50   
22050                     txn_refused                        0    49   
27004                     Base                               0    44   
502                       Base                               0    42   
501                       Base                               0    41   
253                       Base                               0    37   
236                       Base                               0    36   
607                       Base                               0    20   
202                       Insufficient Funds                 0    19   
001                       Base                               0    17   
510                       Base                               0    17   
522                       do_not_honor                       0    17   
27022                     Base                               0    14   
531                       declined                           0    14   
Restricted Card           restricted_card                    0    12   
22028                     txn_refused                        0    10   
302                       attempt_lower_amount               0    10   
233                       Hard Fraud                         0     9   
11453                  

In [98]:
gp = pdf_all.groupby(['failed_response_code', 'failed_response_message'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 100]

success  \
                                                                            sum   
failed_response_code failed_response_message                                      
231                  321: Invalid Merchant                                  102   
0                    27002: Server error, no transaction sent to acq...      74   
27034                System error[] [] [] []                                702   
301                  101: Issuer Unavailable                               1255   
0                    27002: NetGiro server error                           2089   
                     091: No Reply                                         1462   
000                  27002: NetGiro server error                             80   
225                  102: Unable to determine format                        747   
27053                Do not try again/merchant review.[30] [] [11180...    2097   
833                  27053: Do not try again/merchant review                508   
0                    Error validating xml data against the schema on...     101   
27002                Server error, no transaction sent to acquirer[]...     641   
0                    27034: System error                                    285   
233                  27019: Invalid card type                               958   
0                    Litle HTTP response code: 502                         1001   
522                  54: NEW ACCOUNT INFO                                    56   
0                    Communication error, failed to send transType: ...      55   
27019                Invalid Card Type[] [] [] []                          1310   
27033                Acquirer error. Transaction not processed.[null...      49   
0                    510: Unable to Connect to Gateway                      145   
                     Communication error, failed to send transType: ...      48   
11002                No Charge model found[null] [null] [null] [null]       136   
                     No Charge model found[null] [null] [] []              1483   
27001                Transaction refused[30] [] [] []                      1488   
203                  11002: No charge model found                           855   
27020                Invalid Currency Code[null] [null] [null] [null]       164   
23002                Netgiro server error, transaction status unknow...      77   
238                  27020: Invalid currency code                            34   
241                  313: International filtering for issuing card c...     361   
27002                Server error[null] [null] [null] [null]                153   
...                                                                         ...   
27001                Transaction refused[43] [null] [] []                     0   
530                  51: DO NOT HONOR                                         0   
607                  27022: Invalid amount                                    0   
22053                Expired card.[54] [] [000800] [000800]                   0   
530                  27001: Transaction refused[54] [] [] []                  0   
                     27001: Transaction refused[62] [000000] [] []            0   
27001                Transaction refused[05] [] [211820] [211820]             0   
27054                Attempt lower amount.[51] [] [122820] [122820]           0   
27001                Transaction refused[T8] [000000] [] [N]                  0   
530                  27001: Transaction refused[61] [000000] [] []            0   
27001                Transaction refused[13] [000000] [] []                   0   
27083                Insufficient Funds[51] [] [142800] [142800]              0   
                     Insufficient Funds[51] [] [111840] [111840]              0   
522                  522                                                      0   
530                  27001: Transaction refused[62] [null] [] []              0   
27001